In [1]:
#importing packages
import numpy as np
import os
from mido import MidiFile
import sklearn
import joblib
from classes import Note
from classes import File
from classes import Drummer
from functions import extractMidiData

In [2]:
# Load classifier pipeline
classifierPipe = joblib.load("classifierPipe.pkl")

In [3]:
drummerNameToInt = {
    "drummer1": 1,
    "drummer3": 3,
    "drummer4": 4,
    "drummer5": 5,
    "drummer6": 6,
    "drummer7": 7,
    "drummer8": 8,
    "drummer9": 9,
    "drummer10": 10
}

In [4]:
# Extract MIDI information from generatedGroove folder
drummers = os.listdir('./generatedGroove')

extendedDrummersList = []

maxNumberOfFiles = 0

for drummer in drummers:
    if drummer.find('drummer') != -1:
        filesList = []
        files = os.listdir('./generatedGroove/' + drummer)
        for file in files:
            if file.endswith('.mid'):
                maxNumberOfFiles += 1
                filePath = './generatedGroove/' + drummer + "/" + file
                filesList.append(extractMidiData(filePath))

        extendedDrummersList.append(Drummer(drummer, drummerNameToInt[drummer], filesList))

In [5]:
# Get features and labels for each file
numberOfFeatures = 78
feat_test = np.zeros((maxNumberOfFiles, numberOfFeatures)) 
lab_test = np.zeros((maxNumberOfFiles))
fileIndex = 0
for i, drummer in enumerate(extendedDrummersList):
    for file in drummer.files:
        velocities, microtimings = file.extractFeatures()
        if not isinstance(velocities, int):
            feat_test[fileIndex, :] = np.concatenate((velocities, microtimings))
            lab_test[fileIndex] = drummer.num
            fileIndex += 1
            
feat_test = np.resize(feat_test, (fileIndex, numberOfFeatures))
lab_test = np.resize(lab_test, (fileIndex))

#applying the the model on the test data
lab_predict = classifierPipe.predict(feat_test)

In [6]:
# Print final accuracy results and confusion matrix
print('Number of mislabeled samples %d out of %d' % ((lab_test != lab_predict).sum(),lab_test.size))
print('Accuracy:',sklearn.metrics.accuracy_score(lab_test, lab_predict))
print(sklearn.metrics.classification_report(lab_test, lab_predict))
print('confusion matrix')
print(sklearn.metrics.confusion_matrix(lab_test,lab_predict))

Number of mislabeled samples 100 out of 300
Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

         3.0       1.00      1.00      1.00       100
         7.0       0.50      1.00      0.67       100
         9.0       0.00      0.00      0.00       100

    accuracy                           0.67       300
   macro avg       0.50      0.67      0.56       300
weighted avg       0.50      0.67      0.56       300

confusion matrix
[[100   0   0]
 [  0 100   0]
 [  0 100   0]]


/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
